In [28]:
%load_ext autoreload
%autoreload 2
import torch
from src.model.neural_network import MyBertModel, predict, TRANSFORMER_HF_ID
from src.model.data_loading import get_data_loader_from_dataset, get_text_and_labels, embed_inputs
from src.config import config
import numpy as np
import pandas as pd
import time
import plotly.graph_objects as go
import plotly.express as px
from transformers import BertTokenizerFast
import logging

[autoreload of src.model.neural_network failed: Traceback (most recent call last):
  File "g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 309, in update_function
    setattr(old, name, getattr(new, name))
ValueError: predict() requires a code object with 0 free vars, not 2392296783874
]


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Settings
FROM_SCRATCH = True
batch_size = 4
tokenizer = BertTokenizerFast.from_pretrained(TRANSFORMER_HF_ID)
input_col_name = config.model.input_col_name
target_col_name = config.model.target_col_name

In [11]:
dataset = pd.read_parquet(config.data.merged, columns=[input_col_name, target_col_name, "split"])

split = "training"
texts, labels = get_text_and_labels(dataset, split)
inputs, masks = embed_inputs(texts, tokenizer)

func:'embed_inputs' took: 377.3339 sec


In [31]:
model = MyBertModel()
model.load_state_dict(torch.load("data/model"))
model.eval()

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")
model.to(device)

start = time.time()

prediction = predict(model, (inputs, masks), device)
dataset.loc[dataset["split"] == "training", "fcst"] = prediction

end = time.time()
print(f"{start-end:.2f}s")

Using GPU.


OutOfMemoryError: CUDA out of memory. Tried to allocate 574.00 MiB. GPU 0 has a total capacty of 6.00 GiB of which 0 bytes is free. Of the allocated memory 5.00 GiB is allocated by PyTorch, and 251.07 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Regression Task Performance

In [ ]:
test_labels = dataset.loc[:, config.model.target_col_name].tolist()

print(f"Length of evaluation set: {len(y_pred_scaled)}")
print("Vanilla results:")
y_hat = y_pred_scaled
y = np.array(test_labels)
assert len(y_hat) == len(y)

mae, rw_mae, TP, TN = get_metrics(y_hat, y)
metrics_dict = dict(mae=[mae], mae_rw=[rw_mae], TP=[TP], TN=[TN])
metrics_df = pd.DataFrame.from_dict(metrics_dict)
print(metrics_df)


pred_margin_mask = np.abs(y_pred_scaled) >= 0.02

print(f"\nWith prediction margin mask:")
y_hat = y_pred_scaled[pred_margin_mask]
y = np.array(test_labels)[pred_margin_mask]
print(f"\nLength of prediction margin masked evaluation set: {len(y_hat)}")
mae, rw_mae, TP, TN = get_metrics(y_hat, y)
metrics_dict = dict(mae=[mae], mae_rw=[rw_mae], TP=[TP], TN=[TN])
metrics_df = pd.DataFrame.from_dict(metrics_dict)
print(metrics_df)



In [ ]:
##############
# Import stocks
stocks = pd.read_pickle("data/stocks.pkl").reset_index()
# TODO: Do same transformations as import in asset_data_preprocessor


# Analysis of single forecast: 

In [ ]:
idx = 11
tmp = test_dat.loc[pred_margin_mask]
row = tmp.iloc[idx, :]
print(row)
# print(f"Fcst: {row.Fcst}")
# print(f"Target: {row.IntradayReturn}")

print(row.body[:750])
pr_time, ticker, fcst = row[["Date", "ID", "Fcst"]]
df = stocks.query("(Date >= @pr_time) & (ID == @ticker)").head(30)
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
print(row)

# Trading Performance

In [ ]:
tmp = test_dat.loc[pred_margin_mask].dropna()

In [ ]:
tmp.head()

In [ ]:
trades = np.sign(tmp["Fcst"])*tmp["CloseToCloseReturn"]

In [ ]:
trades.mean()

In [ ]:
px.scatter(tmp, x=config.model.target_col_name, y="Fcst")